In [228]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import requests
from yahoo_fin import stock_info
import yfinance as yf
import re
import random
import time

In [30]:
pd.set_option('display.max_rows', 100)

In [2]:
res = requests.get("https://www.sec.gov/Archives/edgar/data/1595888/000159588820000025/xslForm13F_X01/13F-InfoTable.20201116.xml")

In [39]:
def Form_13F_to_dataframe(soup):
    rows = soup.find_all('tr')[11:]
    positions = []
    for row in rows:
        dic = {}
        position = row.find_all('td')
        dic["NAME_OF_ISSUER"] = position[0].text
        dic["TITLE_OF_CLASS"] = position[1].text
        dic["CUSIP"] = position[2].text
        dic["VALUE"] = int(position[3].text.replace(',', ''))
        dic["SHARES"] = int(position[4].text.replace(',', ''))
        dic["PUT/CALL"] = position[6].text
        #dic["DATE"] = date.strip(".html")
        positions.append(dic)

    df = pd.DataFrame(positions)
    return df

def format_13F_df_by_value(df,value):
    
    return df[df['VALUE'] >= value]

In [8]:
soup = BS(res.content,"lxml")

In [44]:
df = Form_13F_to_dataframe(soup)

In [45]:
df_filter = df[df['VALUE'] >= 500000]

In [46]:
df_filter = df_filter[~df_filter['NAME_OF_ISSUER'].str.contains("ETF")].head(100)

In [47]:
df_filter.sort_values('VALUE',ascending=False).head(100)

,NAME_OF_ISSUER,TITLE_OF_CLASS,CUSIP,VALUE,SHARES,PUT/CALL
404,AMAZON COM INC,COM,023135106,11059599,3512400,Call
7871,TESLA INC,COM,88160R101,9059061,21116200,Call
611,APPLE INC,COM,037833100,7014392,60568100,Call
405,AMAZON COM INC,COM,023135106,5521298,1753500,Put
7872,TESLA INC,COM,88160R101,5287720,12325400,Put
4272,INVESCO QQQ TR,COM,46090E103,5004426,18011900,Put
612,APPLE INC,COM,037833100,2390805,20644200,Put
4271,INVESCO QQQ TR,COM,46090E103,2288457,8236600,Call
7415,SPDR GOLD TR,COM,78463V107,2040794,11522100,Call
293,ALIBABA GROUP HLDG LTD,COM,01609W102,1976780,6724200,Call


In [49]:
fintel = requests.get("https://fintel.io/i13f/jane-street-group-llc/2020-09-30-0")

In [50]:
fintel_soup = BS(fintel.content,'lxml')

In [150]:
fintel_table = fintel_soup.findAll('tr')[5:]

In [248]:
def get_fintel_13f_urls(year_month):
    """
    year_month: str of form 'YYYY-MM' e.g '2020-09'
    gets 13f urls filed on that date
    """
    
    sitemap = requests.get("https://fintel.io/investor13f.xml")
    sitemap_soup = BS(sitemap.content, "lxml")
    sitemap_urls = sitemap_soup.findAll('loc')
    sitemap_urls = [i for i in sitemap_urls if year_month in i.text]
    return sitemap_urls
    

def get_tickers_from_13furl(url,fund_name):
    
    user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]
    user_agent = random.choice(user_agent_list)
    #Set the headers 
    headers = {'User-Agent': user_agent}
    #Make the request

    table = requests.get(url,headers=headers)
    soup = BS(table.content, "lxml")
    table = soup.findAll('tr')[6:]
    
    fund_investment_info = {}
    fund_investment_info_sub_dict = {'ticker':{'name':'','curr_val':int(),'curr_shares':}}
    tickers = []
    
    for i in table:

        try:
            cell = i.findAll('td')[0]
            cell = cell.findAll('a')[0]
            ticker = cell.text.split('/')[0]
            comp_name = cell.text.split('/')[1]
            
            curr_val = int(i.findAll('td')[7].text.replace(',',''))
            curr_shares = int(i.findAll('td')[4].text.replace(',',''))
            
            ticker = re.sub('[^0-9a-zA-Z]+', '',ticker)
            #print(ticker)
            if len(ticker) <= 6:
                tickers.append(ticker)
        except:
            pass

    return tickers

def get_tickers_set_from_funds(fund_url_dict):
    """
    input:
    fund_url_dict: maps fund name to fintel 13f url
    
    output:
    returns set of all tickers for these funds. And returns dict of funds to the tickers they invested.
    """
    


    ticker_set = set()
    fund_tickers = {}
    
    for fund in fund_url_dict:
        url = "http://webcache.googleusercontent.com/search?q=cache:" + fund_url_dict[fund]
        tickers = get_tickers_from_13furl(url,fund)
        tickers = set(tickers)
        fund_tickers[fund] = tickers
        ticker_set.update(tickers)
        time.sleep(3)
        
    return ticker_set, fund_tickers
        

In [184]:
ticker_set, fund_tickers = get_tickers_set_from_funds(fund_urls)

In [186]:
ticker_set

512

In [249]:
ticker_set_2, fund_tickers_2 = get_tickers_set_from_funds(fund_urls)

In [250]:
len(ticker_set_2)

3655

In [376]:
fund_tickers_2['Citadel']

set()

In [84]:
top_hedge_funds = requests.get("https://www.pionline.com/interactive/largest-hedge-fund-managers-2020")

In [85]:
top_hedge_funds_soup = BS(top_hedge_funds.content,"lxml")

In [ ]:
hedge_fund_names = []
for i in top_hedge_funds_soup.findAll('tr')[1:]:
    row = i.findAll('td',{'class':'firm'})
    #print(row[0].find('td',{'class':'firm'}))
    row_text = row[0].text
    if row_text[-1].isdigit():
        row_text = row_text[:-1]
    hedge_fund_names.append(row_text)
    #print(row[0].text)
    

In [135]:
urls_to_get = {}

for name in hedge_fund_names:
    name_lower = re.sub('[^0-9a-zA-Z]+', '',name).lower().replace('mgmt','management')
    
    match = [i.text for i in sitemap_urls_2020 if name_lower in re.sub('[^0-9a-zA-Z]+', '',i.text)]
    urls_to_get[name] = match[0] if match else []

In [136]:
urls_to_get

{'Bridgewater Associates': 'https://fintel.io/i13f/bridgewater-associates-lp/2020-09-30-0',
 'Renaissance Technologies': 'https://fintel.io/i13f/renaissance-technologies-llc/2020-09-30-0',
 'Man Group': 'https://fintel.io/i13f/man-group/2020-09-30-0',
 'Millennium Mgmt.': 'https://fintel.io/i13f/millennium-management-llc/2020-09-30-0',
 'Elliott Mgmt.': 'https://fintel.io/i13f/elliott-management/2020-09-30-0',
 'BlackRock ': 'https://fintel.io/i13f/blackrock/2020-09-30-0',
 'Two Sigma Investments/Advisers': [],
 'TCI Fund Mgmt.': 'https://fintel.io/i13f/tci-fund-management/2020-09-30-0',
 'Citadel': 'https://fintel.io/i13f/citadel-advisors-llc/2020-09-30-0',
 'D.E. Shaw Group': [],
 'AQR Capital Mgmt.': 'https://fintel.io/i13f/aqr-capital-management-llc/2020-09-30-0',
 'Davidson Kempner Capital Mgmt.': 'https://fintel.io/i13f/davidson-kempner-capital-management-lp/2020-09-30-0',
 'Farallon Capital Mgmt.': 'https://fintel.io/i13f/farallon-capital-management-llc/2020-09-30-0',
 'Baupost 

In [165]:
len(urls_to_get)

107

In [177]:
fund_urls = {'Bridgewater Associates': 'https://fintel.io/i13f/bridgewater-associates-lp/2020-09-30-0',
 'Renaissance Technologies': 'https://fintel.io/i13f/renaissance-technologies-llc/2020-09-30-0',
 'Man Group': 'https://fintel.io/i13f/man-group/2020-09-30-0',
 'Millennium Mgmt.': 'https://fintel.io/i13f/millennium-management-llc/2020-09-30-0',
 'Elliott Mgmt.': 'https://fintel.io/i13f/elliott-management/2020-09-30-0',
 'BlackRock ': 'https://fintel.io/i13f/blackrock/2020-09-30-0',
 'Two Sigma Investments/Advisers': 'https://fintel.io/i13f/two-sigma-securities-llc/2020-09-30-0',
 'TCI Fund Mgmt.': 'https://fintel.io/i13f/tci-fund-management/2020-09-30-0',
 'Citadel': 'https://fintel.io/i13f/citadel-advisors-llc/2020-09-30-0',
 'D.E. Shaw Group': 'https://fintel.io/i13f/d-e-shaw/2020-09-30-0',
 'AQR Capital Mgmt.': 'https://fintel.io/i13f/aqr-capital-management-llc/2020-09-30-0',
 'Davidson Kempner Capital Mgmt.': 'https://fintel.io/i13f/davidson-kempner-capital-management-lp/2020-09-30-0',
 'Farallon Capital Mgmt.': 'https://fintel.io/i13f/farallon-capital-management-llc/2020-09-30-0',
 'Baupost Group': 'https://fintel.io/i13f/baupost-group-llc-ma/2020-09-30-0',
 'Marshall Wace': 'https://fintel.io/i13f/marshall-wace-llp/2020-09-30-0',
 'Capula Investment Mgmt.': 'https://fintel.io/i13f/capula-management/2020-09-30-0',
 'Canyon Capital': 'https://fintel.io/i13f/steel-canyon-capital/2020-09-30-0',
 'Viking Global Investors ': 'https://fintel.io/i13f/viking-global-investors-lp/2020-09-30-0',
 'Point72 Asset Mgmt.': 'https://fintel.io/i13f/point72-asset-management/2020-09-30-0',
 'York Capital Management': 'https://fintel.io/i13f/york-capital-management-global-advisors-llc/2020-09-30-0',
 'Element Capital Mgmt.': 'https://fintel.io/i13f/element-capital-management-llc/2020-09-30-0',
 'Cevian Capital': 'https://fintel.io/i13f/cevian-capital-ii-gp/2020-09-30-0',
 'GoldenTree Asset Mgmt.': 'https://fintel.io/i13f/goldentree-asset-management-lp/2020-09-30-1',
 'Graham Capital Mgmt.': 'https://fintel.io/i13f/graham-capital-management/2020-09-30-0',
 'Anchorage Capital Group': 'https://fintel.io/i13f/anchorage-capital-group/2020-09-30-0',
 'King Street Capital Mgmt.': 'https://fintel.io/i13f/king-street-capital-management/2020-09-30-0',
 'Angelo Gordon': 'https://fintel.io/i13f/angelo-gordon/2020-09-30-0',
 'D1 Capital Partners': 'https://fintel.io/i13f/d1-capital-partners/2020-09-30-0',
 'ExodusPoint Capital': 'https://fintel.io/i13f/exoduspoint-capital-management/2020-09-30-0',
 'Pershing Square': 'https://fintel.io/i13f/pershing-square-capital-management/2020-09-30-0',
 'Lone Pine Capital': 'https://fintel.io/i13f/lone-pine-capital-llc/2020-09-30-0',
 'Tudor Investment': 'https://fintel.io/i13f/tudor-investment-corp-et-al/2020-09-30-0',}

In [178]:
len(fund_urls)

32

In [176]:
[i.text for i in sitemap_urls_2020 if 'sculptor' in i.text]

[]

In [187]:
corr_tickers = pd.read_csv('ALL_STOCKS_CORR.csv')

In [198]:
corr_tickers = set(list(corr_tickers['ticker1']) + list(corr_tickers['ticker2']))

In [258]:
ticker_sector_df = pd.read_csv('ticker sector df.csv')
ticker_sector_df = ticker_sector_df[['sector','ticker']]

In [255]:
tickers_to_add = [i for i in ticker_set_2 if i not in corr_tickers and i not in ticker_sector_df['ticker']]

In [256]:
len(tickers_to_add)

1291

In [257]:
ticker_sector = {"ticker":[],"sector":[]}
failed = 0
for ticker in tickers_to_add:
    print(ticker)
    try:
        sector = yf.Ticker(ticker).info['sector']
    
        ticker_sector['ticker'].append(ticker)
        ticker_sector['sector'].append(sector)
    except:
        print("Failed : ", ticker)
        failed += 1

DOO
Failed :  DOO
MAA
IGOV
Failed :  IGOV
IVE
Failed :  IVE
BDX
VIAC
AKRO
VCR
Failed :  VCR
OIBR3
Failed :  OIBR3
IRET
Failed :  IRET
CVAC
SQM
FANH
TLRY
ESS
RJI
Failed :  RJI
CS
KTB
CLNC
HUYA
PK
IGF
Failed :  IGF
ARCT
DBEU
Failed :  DBEU
DFEB
Failed :  DFEB
PSQ
Failed :  PSQ
SECO
EXPCU
WRI
SOGO
BZUN
MYOK
Failed :  MYOK
DDM
Failed :  DDM
GOL
SPIB
Failed :  SPIB
IWS
Failed :  IWS
SHV
Failed :  SHV
FDN
Failed :  FDN
WORK
AUTL
BGNE
SPSB
Failed :  SPSB
ROSEW
Failed :  ROSEW
FNTCW
Failed :  FNTCW
SCHX
Failed :  SCHX
AMCIW
Failed :  AMCIW
NNN
SVACU
TRTX
CRSR
OSTK
FOUR
APG
EEM
Failed :  EEM
TMDX
REG
ICLK
CWT
NVAX
AMK
SCHE
Failed :  SCHE
YJ
RPT
CUBE
TUFN
BFB
Failed :  BFB
GSK
DDOG
XLB
Failed :  XLB
GUNR
Failed :  GUNR
GWPH
INSUW
Failed :  INSUW
RESI
CHWY
BEPC
NCT
Failed :  NCT
BRKB
Failed :  BRKB
AMRN
ADSW
Failed :  ADSW
WMG
LEVI
NARI
MSGE
CIM
TCBIP
ILPT
FEAC
Failed :  FEAC
VTIP
Failed :  VTIP
FSLY
BXP
CRWD
BMQZF
Failed :  BMQZF
EWZ
Failed :  EWZ
EWW
Failed :  EWW
MNR
IYE
Failed :  IYE
EXPC
HDS

STSA
SPYG
Failed :  SPYG
BIF
BLI
JD
BRMK
WELL
SHM
Failed :  SHM
RST
Failed :  RST
MSBF
Failed :  MSBF
MOGA
Failed :  MOGA
BCS
MR
Failed :  MR
SOXL
Failed :  SOXL
NOAH
OMP
GLDM
Failed :  GLDM
DTIL
DRD
UNG
Failed :  UNG
GNL
NIU
UHT
XSLV
Failed :  XSLV
DFPH
MBT
NEW
USO
Failed :  USO
OACU
Failed :  OACU
CLDT
IRT
CFFA
Failed :  CFFA
GPMT
HMIN
Failed :  HMIN
BP
CX
LOGN
RLJ
CALM
LTC
SDS
Failed :  SDS
MOMO
ADAP
MNRL
HBHC
Failed :  HBHC
MIICF
Failed :  MIICF
SINT
ARYBU
Failed :  ARYBU
BTI
SWTX
TCBWS
Failed :  TCBWS
WABC
TS
OLP
NOVSW
Failed :  NOVSW
PBR
VSIGX
Failed :  VSIGX
SBG
XLI
Failed :  XLI
FVD
Failed :  FVD
TRIL
DPHCU
Failed :  DPHCU
DXJ
Failed :  DXJ
SPG
SCPL
FCG
Failed :  FCG
TM
CODX
BRP
FULC
EWY
Failed :  EWY
SLYV
Failed :  SLYV
BLD
BHR
UE
JMST
Failed :  JMST
CCIPRA
Failed :  CCIPRA
GLPG
PLTR
FVAC
Failed :  FVAC
BTAI
KOF
LMRK
AEG
CCH
Failed :  CCH
IEI
Failed :  IEI
ZBPRA
Failed :  ZBPRA
FLML
Failed :  FLML
PRVL
SE
FAMI
FSCT
Failed :  FSCT
IQ
KDNY
LUMN
TFI
Failed :  TFI
CDR
NVS
DEI
SMH


In [261]:
ticker_sector_df = ticker_sector_df.append(pd.DataFrame(ticker_sector),ignore_index=True)

In [263]:
ticker_sector_df.to_csv("ticker sector df new.csv")

In [252]:
fund_tickers_2['Tudor Investment']

{'MNST',
 'MAA',
 'NATI',
 'TT',
 'KR',
 'IBKR',
 'ASAN',
 'BDX',
 'PKI',
 'M',
 'IIVI',
 'EXH',
 'HLF',
 'WCC',
 'LKQ',
 'CLF',
 'BILL',
 'LMND',
 'HRL',
 'USM',
 'SPPI',
 'LOGM',
 'EFX',
 'VOD',
 'BIIB',
 'GME',
 'JBLU',
 'VSAT',
 'TA',
 'AMAG',
 'TLRY',
 'FMBI',
 'SSYS',
 'MRC',
 'FR',
 'ESS',
 'GPS',
 'WIRE',
 'SNA',
 'NLOK',
 'KTB',
 'DQ',
 'CLNC',
 'PII',
 'MCHP',
 'SFL',
 'CVBF',
 'VRM',
 'ETM',
 'DCOM',
 'BR',
 'RARE',
 'WKHS',
 'NCR',
 'ACRS',
 'SWCH',
 'OFC',
 'ARCT',
 'LNT',
 'SILK',
 'STLD',
 'WRLD',
 'WRI',
 'TRGP',
 'EEFT',
 'AIMT',
 'RGA',
 'KMPR',
 'AXS',
 'AGS',
 'MYOK',
 'PING',
 'ETFC',
 'KMI',
 'PNW',
 'INDB',
 'PLNT',
 'NTCT',
 'CHUY',
 'TFX',
 'PSX',
 'TGH',
 'ENVA',
 'NBTB',
 'PEGA',
 'WORK',
 'CDW',
 'SR',
 'HVTA',
 'THC',
 'AGCO',
 'AUTL',
 'SWKS',
 'TOWN',
 'VNE',
 'PNM',
 'CNP',
 'BECN',
 'ENS',
 'ETR',
 'HAL',
 'ETH',
 'NXPI',
 'IFF',
 'AMCX',
 'DLB',
 'ABBV',
 'MTB',
 'TRUE',
 'HELE',
 'ENV',
 'PLYM',
 'HBAN',
 'BGCP',
 'COTY',
 'CVI',
 'ZEN',
 'AMH',
 'SCI

In [227]:
import random
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]
    user_agent = random.choice(user_agent_list)
    #Set the headers 
    headers = {'User-Agent': user_agent}
    #Make the request
    response = requests.get(url,headers=headers)

In [349]:
res = requests.get('http://webcache.googleusercontent.com/search?q=cache:https://fintel.io/i13f/bridgewater-associates-lp/2020-09-30-0')

In [350]:
res.content

b'<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><base href="https://fintel.io/i13f/bridgewater-associates-lp/2020-09-30-0"><style>body{margin-left:0;margin-right:0;margin-top:0}#bN015htcoyT__google-cache-hdr{background:#f8f9fa;font:13px arial,sans-serif;text-align:left;color:#202124;border:0;margin:0;border-bottom:1px solid #dadce0;line-height:16px;padding:16px 28px 24px 28px}#bN015htcoyT__google-cache-hdr *{display:inline;font:inherit;text-align:inherit;color:inherit;line-height:inherit;background:none;border:0;margin:0;padding:0;letter-spacing:0}#bN015htcoyT__google-cache-hdr a{text-decoration:none;color:#1967D2}#bN015htcoyT__google-cache-hdr a:hover{text-decoration:underline}#bN015htcoyT__google-cache-hdr a:visited{color:#4b11a8}#bN015htcoyT__google-cache-hdr div{display:block;margin-top:4px}#bN015htcoyT__google-cache-hdr b{font-weight:bold;display:inline-block;direction:ltr}</style><div id="bN015htcoyT__google-cache-hdr"><div><span>This is Google\'s cache of <a

In [351]:
soup = BS(res.content,"lxml")

In [365]:
soup.findAll('tr')[6:][0].findAll('td')

[<td><a href="/so/us/pm">PM / Philip Morris International, Inc.</a>
 </td>,
 <td><a href="/so/us/pm/bridgewater-associates-lp" target="_blank"><span class="fa fa-external-link"></span></a></td>,
 <td align="right"></td>,
 <td align="right"> 45,752</td>,
 <td align="right"> 0</td>,
 <td align="right"> -100.00</td>,
 <td align="right"> 3,205</td>,
 <td align="right"> 0</td>,
 <td align="right"> -100.00</td>]

In [374]:
for i in soup.findAll('tr')[6:]:
    cell = i.findAll('td')[0]
    cell = cell.findAll('a')[0]
    print(cell.text)
    print(int(i.findAll('td')[7].text.replace(',','')))
    print(int(i.findAll('td')[4].text.replace(',','')))

PM / Philip Morris International, Inc.
0
0
AEP / American Electric Power Company, Inc.
0
0
EQR / Equity Residential
0
0
ORLY / O'Reilly Automotive, Inc.
0
0
AWK / American Water Works Co., Inc.
0
0
BAM / Brookfield Asset Management, Inc.
540
16386
CPB / Campbell Soup Co.
0
0
LDOS / Leidos Holdings, Inc.
0
0
XPEV / XPeng Inc. American depositary shares, each representing two Class A ordinary shares
473
23589
INDA / iShares MSCI India ETF
0
0
872590112 / T-Mobile US Inc
0
0
SPGI / S&P Global Inc.
4782
13261
HELE / Helen of Troy Ltd.
2528
13065
COST / Costco Wholesale Corp.
77625
218662
ATR / AptarGroup, Inc.
1630
14399
GNTX / Gentex Corp.
1645
63895
SINA / Sina Corp.
2017
47339
LH / Laboratory Corp. of America Holdings
4372
23224
MPWR / Monolithic Power Systems, Inc.
955
3414
NUE / Nucor Corp.
5590
124620
CRI / Carter's, Inc.
1003
11588
GIS / General Mills, Inc.
18728
303625
KMX / CarMax, Inc.
3922
42673
AGCO / AGCO Corp.
440
5931
FSLR / First Solar, Inc.
1233
18619
IFF / International F

22860
GH / Guardant Health, Inc.
2668
23871
TCOM / Trip.com Group Limited
30932
993306
JNPR / Juniper Networks, Inc.
712
33134
QDEL / QUIDEL Corp.
2347
10697
WAB / Wabtec Corp.
935
15112
NYT / New York Times Co.
2907
67936
ACM / AECOM
2766
66104
CLX / Clorox Company (The)
13249
63041
MNST / Monster Beverage Corporation
19840
247383
PG / Procter & Gamble Co. (The)
170329
1225476
ETN / Eaton Corporation
3146
30838
GPC / Genuine Parts Co.
3647
38322
QCOM / QUALCOMM Incorporated
1016
8633
AMKR / Amkor Technology, Inc.
885
79010
TAP / Molson Coors Brewing Co.
3138
93504
COLM / Columbia Sportswear Co.
744
8552
VIAV / Viavi Solutions Inc.
626
53408
LQD / iShares iBoxx $ Investment Grade Corporate Bond ETF
146380
1086629
BSTI / BEST Inc.
1183
394264
TSN / Tyson Foods, Inc.
10581
177887
CMI / Cummins, Inc.
2439
11551
ZNGA / Zynga Inc.
3562
390559
TFX / Teleflex, Inc.
5248
15415
KMB / Kimberly-Clark Corp.
24906
168674
EWZ / iShares MSCI Brazil ETF
4637
167647
PHM / PulteGroup, Inc.
3296
71207
BE

In [ ]:
#### get correlations of new stocks
#### Get more info on funds
### remember to include properties for relationships between funds and stocks

In [264]:
import PriceData

In [265]:
price_data = PriceData.create_price_table(tickers_to_add)

In [269]:
price_data = price_data[[i for i in price_data.columns if i in ticker_sector['ticker']]]

In [270]:
price_data = PriceData.format_price_table(price_data)

In [272]:
price_data = PriceData.format_price_table_nan_values(price_data)

In [274]:
stdevs,price_data_not_null = PriceData.get_stdevs_trim_price_data(price_data)

In [277]:
price_corr = PriceData.get_correlation_table(price_data_not_null)
price_corr = PriceData.trim_cor_table(price_corr)

In [278]:
price_corr

,ticker1,ticker2,cor
1824,MAA,ESS,0.858375
2280,MAA,CS,0.586229
2281,SQM,CS,0.641049
2284,ESS,CS,0.566711
2736,MAA,CLNC,0.516042
...,...,...,...
207891,HIW,LAND,0.602223
207903,CHCT,LAND,0.515761
207909,VICI,LAND,0.534914
207912,DEA,LAND,0.538639


In [279]:
price_corr.to_csv("fund stocks corr.csv")

In [280]:
price_data_not_null.to_csv("fund price data.csv")

In [281]:
ALL_STOCKS_price_data_not_null = pd.read_csv("ALL_STOCKS_price_data_not_null.csv")

In [282]:
ALL_STOCKS_price_data_not_null.append(price_data_not_null)

,Unnamed: 0,LTHM,EPZM,HQY,ICBK,SIEN,TBK,IRTC,HLNE,PCSB,...,PCSA,AEP,FSR,HUSN,LIXT,EYPT,WRAP,LRN,REPL,ISIG
0,2019-01-02,13.990000,6.67,59.110001,17.206980,13.64,29.850000,69.330002,34.847992,19.305901,...,24.500000,68.427811,9.63,0.900,0.94,19.50,3.24,23.969999,10.500000,10.71
1,2019-01-03,13.590000,5.98,54.900002,17.167959,13.63,29.639999,65.440002,34.597977,19.453125,...,25.549999,68.268311,9.63,0.900,0.94,19.10,3.25,23.930000,10.530000,10.22
2,2019-01-04,14.260000,7.94,58.000000,17.626421,13.94,30.320000,68.820000,35.501865,19.943869,...,25.549999,68.896873,9.63,1.010,0.94,22.60,3.25,24.100000,10.550000,11.34
3,2019-01-07,14.400000,8.40,53.490002,17.548389,14.19,30.660000,72.809998,35.136471,19.492384,...,25.200001,68.512245,9.63,1.100,0.94,22.90,3.12,24.260000,10.990000,11.34
4,2019-01-08,14.350000,9.08,52.860001,18.406788,14.44,31.440001,73.739998,35.021076,19.865351,...,24.500000,69.356567,9.65,1.120,0.99,22.10,3.05,24.280001,10.260000,10.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,2020-12-29,17.780001,11.49,69.110001,22.709999,3.92,48.590000,242.179993,77.470001,16.290001,...,6.430000,81.180000,15.45,3.350,3.14,5.41,4.92,21.059999,40.830002,7.14
503,2020-12-30,18.620001,11.57,69.470001,22.400000,3.96,49.080002,235.399994,76.870003,15.860000,...,6.750000,81.510002,15.36,3.720,3.09,5.90,4.92,21.580000,42.340000,6.44
504,2020-12-31,18.840000,10.86,69.709999,22.080000,3.89,48.549999,237.210007,78.050003,15.940000,...,6.595000,83.269997,14.65,3.620,3.17,6.58,4.83,21.230000,38.150002,5.88
505,2021-01-04,18.780001,10.71,68.910004,20.709999,3.87,49.110001,227.970001,75.739998,15.970000,...,6.630000,81.540001,15.19,3.710,3.21,9.01,4.80,21.010000,38.230000,7.31


In [290]:
ALL_STOCKS_price_data_not_null.index = ALL_STOCKS_price_data_not_null['Unnamed: 0']
ALL_STOCKS_price_data_not_null.index.name = 'ticker'
del ALL_STOCKS_price_data_not_null['Unnamed: 0']

In [298]:
price_data_not_null

ticker,MAA,SQM,FANH,TLRY,ESS,CS,CLNC,HUYA,PK,ARCT,...,HDB,PSHG,STWD,INFY,TLND,H,FET,PSO,WNS,LAND
2019-01-02,86.617706,35.898205,20.527796,70.459999,222.102921,10.505462,13.559259,15.770000,22.598711,4.690000,...,51.472233,0.725769,16.042101,9.084573,37.720001,66.127518,88.800003,11.164757,41.099998,10.766399
2019-01-03,87.244080,34.705322,19.957331,70.000000,223.276367,10.419819,13.490603,15.670000,22.616392,4.850000,...,50.553711,0.715962,16.017700,8.952227,37.200001,63.767231,90.199997,11.192950,39.509998,10.738794
2019-01-04,87.253426,36.382816,20.618349,69.949997,223.708221,10.762389,13.911114,17.110001,23.226454,5.130000,...,51.427551,0.725769,16.261747,9.103479,38.279999,66.344788,96.599998,11.484287,41.680000,11.014855
2019-01-07,87.131889,36.792862,21.098267,71.900002,224.637573,10.838515,14.039840,18.500000,23.624315,5.280000,...,51.164406,0.784615,16.424446,9.179106,39.299999,66.769432,103.599998,11.625256,42.580002,11.088471
2019-01-08,88.889481,38.116219,20.536854,83.260002,230.251434,10.990768,14.460349,18.139999,24.711813,5.410000,...,51.541744,0.784615,16.603415,9.216917,38.970001,67.806389,105.199997,12.048163,43.590000,11.300119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-04,122.709999,51.099998,12.580000,9.000000,228.250000,12.890000,7.090000,20.600000,16.530001,48.900002,...,70.000000,4.640000,18.520000,17.250000,40.290001,71.000000,12.750000,9.190000,70.540001,14.290000
2021-01-05,122.669998,55.009998,12.610000,9.510000,227.220001,13.170000,7.320000,21.820000,16.870001,48.790001,...,71.540001,4.630000,18.340000,17.650000,41.279999,71.550003,13.060000,9.140000,70.620003,14.280000
2021-01-06,124.699997,57.389999,12.890000,10.770000,236.669998,13.790000,8.060000,20.440001,17.660000,49.369999,...,70.779999,4.560000,18.980000,17.730000,41.279999,74.000000,13.260000,9.510000,71.300003,14.360000
2021-01-07,124.550003,59.450001,13.100000,11.340000,231.500000,14.100000,8.210000,20.910000,17.430000,66.970001,...,71.959999,4.860000,18.799999,17.549999,41.040001,74.540001,13.930000,9.230000,71.669998,14.590000


In [301]:
price_data_join = price_data_not_null.join(ALL_STOCKS_price_data_not_null)

In [299]:
[i for i in price_data_not_null.columns if i in ALL_STOCKS_price_data_not_null.columns]

[]

In [305]:
price_data_join = price_data_join[:-3]

In [314]:
price_data_join.columns.name = 'ticker'

In [318]:
price_data_join_corr = PriceData.get_correlation_table(price_data_join)
price_data_join_corr = PriceData.trim_cor_table(price_data_join_corr)

In [319]:
price_data_join_corr

,ticker1,ticker2,cor
15884,MAA,ESS,0.859537
19855,MAA,CS,0.587596
19856,SQM,CS,0.637898
19859,ESS,CS,0.568247
23826,MAA,CLNC,0.515604
...,...,...,...
15736897,PFE,AEP,0.505394
15736969,LHX,AEP,0.560803
15736996,RE,AEP,0.522803
15737016,J,AEP,0.520341


In [321]:
df_ticker_nodes = pd.DataFrame()
set_tickers = set(list(price_data_join_corr['ticker1']) + list(price_data_join_corr['ticker2']))
df_ticker_nodes['ticker'] = list(set_tickers)

In [322]:
df_ticker_nodes

,ticker
0,NVTA
1,MNST
2,MAA
3,TXRH
4,BG
...,...
2746,SASR
2747,CVGI
2748,EW
2749,BJRI


In [323]:
### New ticker to add ####
# not all new tickers will have corr relationship to other tickers
df_all_new_ticker_nodes = pd.DataFrame()
df_all_new_ticker_nodes['ticker'] = price_data_not_null.columns
#####
### Get Names for these tickers - maybe use API
### merge with sector
### upload these tickers as nodes
### upload relationships

### Need to get info on stock positions - add into functions above
### create nodes from the funds
### create relationships between funds and tickers with position properties.

In [328]:
## GET TICKER NAMES
ticker_names = {"ticker":[],"name":[]}
failed = 0
for ticker in df_all_new_ticker_nodes['ticker']:
    print(ticker)
    try:
        name = yf.Ticker(ticker).info['longName']
    
        ticker_names['ticker'].append(ticker)
        ticker_names['name'].append(name)
    except:
        print("Failed : ", ticker)
        failed += 1


MAA
SQM
FANH
TLRY
ESS
CS
CLNC
HUYA
PK
ARCT
SECO
WRI
SOGO
BZUN
GOL
AUTL
BGNE
NNN
TRTX
OSTK
REG
ICLK
RPT
CUBE
GWPH
RESI
AMRN
CIM
ILPT
BXP
MNR
BIDU
QIWI
ERJ
VODPF
SOHU
ELP
AMT
RIO
EXPI
RYN
ALX
VLRS
SWBI
VNET
FMX
IIPR
CHU
WSR
DOC
CCU
SHO
INVH
JBGS
SAN
JOBS
LAMR
KBR
SAP
ESRT
BTO
FRT
HST
KOD
CORR
VALE
ACB
HMC
DRH
ENPH
VNO
BNGO
ZTO
QTS
COLD
WPG
ENIC
FCPT
PSA
HEXO
MSC
PLD
CHL
TGS
HASI
BBVA
GFI
KB
BSM
KYN
NCSM
LKCO
HMY
AZUL
ENZ
NLY
MFA
BCH
MMP
EXR
ZLAB
TWST
GDS
ACH
KT
KMF
EPR
AGNC
TPL
PHTCF
CRTO
FLY
GMLP
PKX
PDM
ARR
XHR
DAVA
LXP
TEVA
LKFN
DX
DL
HPP
BPMP
CXW
UXIN
YY
INN
BSMX
BEP
WB
DRE
ORTX
JQC
VTR
INO
AFIN
DBVT
AAT
CIO
CODI
CBD
LFC
RWT
CELH
SBAC
VIOT
MVIS
KRC
BRG
UDR
UGP
BBU
ACRE
MTL
EPD
SUN
AKR
LSXMA
CETX
FSP
DLR
IVR
O
UBA
GEL
BRFS
AVAL
WPC
LYG
LITB
AVB
CPLG
KREF
TSLA
AHH
SBSW
IEP
FWONK
LSXMK
VERI
AWH
ASRT
DXLG
SRC
TCOM
EGP
IBIO
PCH
BHP
NKLA
WRE
AIV
OUT
SNE
ELS
SAVA
CIB
PRTS
JILL
STOHF
GLPI
PMT
LPL
FMS
TME
TAL
FPI
REXR
APVO
CCLP
FLGT
CEN
PSXP
VOD
FR
DQ
OFC
JKS
MRSN
HT
CLI
ZNH
ASR
PLYM
AMH
GEO


In [329]:
df_ticker_names = pd.DataFrame(ticker_names)

In [330]:
df_ticker_names

,ticker,name
0,MAA,"Mid-America Apartment Communities, Inc."
1,SQM,Sociedad Química y Minera de Chile S.A.
2,FANH,Fanhua Inc.
3,TLRY,"Tilray, Inc."
4,ESS,"Essex Property Trust, Inc."
...,...,...
451,H,Hyatt Hotels Corporation
452,FET,"Forum Energy Technologies, Inc."
453,PSO,Pearson plc
454,WNS,WNS (Holdings) Limited


In [332]:
df_all_new_ticker_nodes = df_all_new_ticker_nodes.merge(df_ticker_names,how='left',on='ticker')

In [345]:
df_all_new_ticker_nodes = df_all_new_ticker_nodes.merge(ticker_sector_df,how='left',on='ticker').drop_duplicates(subset=['ticker'])